# 事前処理

jumanとknpをインストールした後に下記でparse後のファイルを作成した

```
while read line
do
  echo $line | jumanpp | knp -tab >> ./ai.ja/ai.ja.parsed
done < /ai.ja/ai.ja.txt
```

In [1]:
with open("../ai.ja/ai.ja.parsed") as f:
    texts = f.read()

# 40. 係り受け解析結果の読み込み（形態素）

In [3]:
from pydantic import BaseModel
from typing import List

class Morph(BaseModel):
    surface: str
    base: str
    pos: str
    pos1: str

In [6]:
# EOSごとに区切った上で、 \n で区切る
sentences = list(map(lambda x: x.split("\n"), texts.split("EOS")))
# 空の要素削除
sentences = [[element for element in sentence if element != ""] for sentence in sentences]
sentences[:1]

[['# S-ID:1 KNP:4.19-CF1.1 DATE:2021/06/05 SCORE:-1.74640',
  '* -1D <文頭><文末><体言><用言:判><体言止><レベル:C><区切:5-5><ID:（文末）><裸名詞><提題受:30><主節><状態述語><正規化代表表記:人工/じんこう+知能/ちのう><主辞代表表記:知能/ちのう>',
  '+ 1D <文節内><係:文節内><文頭><体言><名詞項候補><先行詞候補><正規化代表表記:人工/じんこう>',
  '人工 じんこう 人工 名詞 6 普通名詞 1 * 0 * 0 "代表表記:人工/じんこう カテゴリ:抽象物" <代表表記:人工/じんこう><カテゴリ:抽象物><正規化代表表記:人工/じんこう><文頭><漢字><かな漢字><名詞相当語><自立><内容語><タグ単位始><文節始>',
  '+ -1D <文末><体言><用言:判><体言止><レベル:C><区切:5-5><ID:（文末）><裸名詞><提題受:30><主節><状態述語><判定詞><名詞項候補><先行詞候補><正規化代表表記:知能/ちのう><用言代表表記:知能/ちのう><Wikipedia上位語:基礎技術><時制-無時制><格解析結果:知能/ちのう:判0:ガ/U/-/-/-/-;トスル/U/-/-/-/-;外の関係/U/-/-/-/->',
  '知能 ちのう 知能 名詞 6 普通名詞 1 * 0 * 0 "代表表記:知能/ちのう カテゴリ:抽象物" <代表表記:知能/ちのう><カテゴリ:抽象物><正規化代表表記:知能/ちのう><文末><Wikipedia上位語:基礎技術:0-1><表現文末><漢字><かな漢字><名詞相当語><自立><複合←><内容語><タグ単位始><文節主辞>']]

In [61]:
# 冒頭の説明文をMorphsにする
morphs = []
for result in sentences[1]:
    if result.startswith(("#", "*", "+")):
        continue
    else:
        elements = result.split()
        morph = Morph(surface=elements[0], base=elements[2], pos=elements[3], pos1=elements[5])
        morphs.append(morph)

In [63]:
morphs[:10]

[Morph(surface='人工', base='人工', pos='名詞', pos1='普通名詞'),
 Morph(surface='知能', base='知能', pos='名詞', pos1='普通名詞'),
 Morph(surface='（', base='（', pos='特殊', pos1='括弧始'),
 Morph(surface='じんこう', base='じんこう', pos='名詞', pos1='普通名詞'),
 Morph(surface='ちのう', base='ちのう', pos='名詞', pos1='普通名詞'),
 Morph(surface='、', base='、', pos='特殊', pos1='読点'),
 Morph(surface='、', base='、', pos='特殊', pos1='読点'),
 Morph(surface='AI', base='AI', pos='名詞', pos1='組織名'),
 Morph(surface='〈', base='〈', pos='特殊', pos1='括弧始'),
 Morph(surface='エーアイ', base='エーアイ', pos='名詞', pos1='組織名')]

# 41. 係り受け解析結果の読み込み（文節・係り受け）
- [KNP:格解析結果書式](https://nlp.ist.i.kyoto-u.ac.jp/?KNP/%E6%A0%BC%E8%A7%A3%E6%9E%90%E7%B5%90%E6%9E%9C%E6%9B%B8%E5%BC%8F)
- [KNPの基本的な出力の読み方](http://cr.fvcrc.i.nagoya-u.ac.jp/~sasano/knp/format.html)

In [5]:
class Chunk(BaseModel):
    morphs: List[Morph]
    dst: List[int]
    srcs: List[int]
    

In [7]:
sentences[2]

['# S-ID:1 KNP:4.19-CF1.1 DATE:2021/06/05 SCORE:-18.26693',
 '* 1D <文頭><括弧始><括弧終><助詞><連体修飾><体言><係:ノ格><区切:0-4><正規化代表表記:日本/にほん+大/だい+百科/ひゃっか+全書/ぜんしょ+(/(+ニッポニカ/ニッポニカ+)/)><主辞代表表記:)/)>',
 '+ 1D <文節内><係:文節内><文頭><地名疑><括弧始><引用内文頭><体言><名詞項候補><先行詞候補><正規化代表表記:日本/にほん><NE内:OTHER>',
 '『 『 『 特殊 1 括弧始 3 * 0 * 0 NIL <文頭><記英数カ><英記号><記号><括弧始><括弧><接頭><非独立接頭辞><タグ単位始><文節始>',
 '日本 にっぽん 日本 名詞 6 地名 4 * 0 * 0 "代表表記:日本/にほん 地名:国" <代表表記:日本/にほん><地名:国><正規化代表表記:日本/にほん><品曖><ALT-日本-にほん-日本-6-4-0-0-"代表表記:日本/にほん 地名:国"><品曖-地名><漢字><かな漢字><名詞相当語><自立><内容語><固有キー><NE:ARTIFACT:B>',
 '+ 2D <文節内><係:文節内><体言><一文字漢字><名詞項候補><先行詞候補><正規化代表表記:大/だい><NE内:ARTIFACT>',
 '大 だい 大 名詞 6 普通名詞 1 * 0 * 0 "代表表記:大/だい 漢字読み:音 組織名末尾 カテゴリ:抽象物" <代表表記:大/だい><漢字読み:音><組織名末尾><カテゴリ:抽象物><正規化代表表記:大/だい><漢字><かな漢字><名詞相当語><自立><複合←><内容語><タグ単位始><固有修飾><NE:ARTIFACT:I>',
 '+ 3D <文節内><係:文節内><体言><名詞項候補><先行詞候補><正規化代表表記:百科/ひゃっか><NE内:ARTIFACT>',
 '百科 ひゃっか 百科 名詞 6 普通名詞 1 * 0 * 0 "代表表記:百科/ひゃっか カテゴリ:抽象物 ドメイン:教育・学習" <代表表記:百科/ひゃっか><カテゴリ:抽象物><ドメイン:教育・学習><正規化代表表記:百科/ひゃっか><漢字><かな漢字><名詞相当